In [7]:
import pandas as pd
data = pd.read_csv("sigma.csv")
# null
data = data.drop_duplicates()
data.duplicated().sum()



0

In [8]:
data

,label,text
0,Ismail Han,😺
1,Ismail Han,Kaz ta hoilo
2,Ismail Han,Kotha na boijai
3,Ismail Han,Ami vavlam tmi ethan
4,Ismail Han,What's up?
...,...,...
106391,Tasin Mahatab,Ata sikis
106392,Tasin Mahatab,V .V.T question
106393,Tasin Mahatab,Most of the important of reproduction😆Ismail Han
106398,Tasin Mahatab,Hu


#### 1st e word tokenize
##### puncuation remove
#### remove special char
#### remove stop word
#### Lemmatization
#### Label Encoder

In [9]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

data['text'] = data['text'].fillna('') # nan(not a number) also kind of NONE type thing. ekhane amra oi value gula change kore dibo
data['text'].apply(lambda x: word_tokenize(x))


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/home/ismail/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# remove punc
import string
def remove_punc(text):
  trans = str.maketrans('', '', string.punctuation)
  return text.translate(trans)

data['text'] = data['text'].apply(remove_punc)
data['text']

In [ ]:
### remove special char (abcd er baire habijabi)
import re
def remove_noise(text):
  t = re.sub('[^a-zA-Z]', ' ', text)
  return t
data['text'] = data['text'].apply(remove_noise)
data['text']

In [ ]:
### remove stop word
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

sw = stopwords.words('english')
def remove_sws(text):
    # low char
    text = text.lower()
    non_stop_word = []
    split_text = text.split()
    for new in split_text:
        if new not in sw:
            non_stop_word.append(new)
            # from nltk.stem import WordNetLemmatizer
            # lemmatizer = WordNetLemmatizer()
            # new = lemmatizer.lemmatize(new)
            # non_stop_word.append(new)
    return " ".join(non_stop_word)
            
# def remove_sws(text):
#   s = [word.lower() for word in text.split() if word.lower() not in sw]
#   return " ".join(s)
data['text'] = data['text'].apply(remove_sws)
data['text']

In [ ]:
### Lemmatization
# similar word
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemma(text):
    
    lemmatize_Word = []
    split_text = text.split()
    for new in split_text:
        if new not in sw:
            lemmatize_Word.append(new)
            from nltk.stem import WordNetLemmatizer
            lemmatizer = WordNetLemmatizer()
            new = lemmatizer.lemmatize(new)
            lemmatize_Word.append(new)
    return " ".join(lemmatize_Word)
           
def lemma(text):
  l = [lemmatizer.lemmatize(word) for word in text.split()]
  return " ".join(l)

data['text'] = data['text'].apply(lemma)
data['text']

In [ ]:
### Label Encoder
# from sklearn.preprocessing import LabelEncoder
# encoder =LabelEncoder()

# data['v1']=encoder.fit_transform(data['v1'])
# data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=3000)
x = tf.fit_transform(data['text']).toarray()
y = data['label']
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x)
from sklearn.naive_bayes import  MultinomialNB
# Create the instance of Naive Bayes
clf = MultinomialNB()
# Fit the data
clf.fit(X_train, y_train)
# Making prediction



y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))



# your_sen = ["join the meeting"]
# your_sen = tf.transform(your_sen).toarray()
# y_pred = clf.predict(X_test)
# print(y_pred)
# print("Accuracy Score: ", accuracy_score(y_test, y_pred))

In [ ]:
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()  # Lowercase
    text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# tokeize korte hobe
your_sen = ["join and love"]

# Preprocess the new sentence before vectorizing
your_sen = [preprocess_text(sen) for sen in your_sen]

# Transform the new sentence using the same vectorizer
your_sen_transformed = tf.transform(your_sen).toarray()
#data['v2'].apply(lambda x: word_tokenize(x))
# Predict the label for the new sentence
your_sen_pred = clf.predict(your_sen_transformed)

# Print the prediction for the new sentence
print("Prediction for your sentence:", 'Spam' if your_sen_pred[0] == 1 else 'Ham')





